In [15]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00


In [14]:
!pip install accelerate -U
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
import pandas as pd


filtered_df = pd.read_excel('/content/Лист Microsoft Excel.xlsx')

data = filtered_df

# Преобразование в формат "вопрос-ответ"
formatted_data = []
for index, row in data.iterrows():
    input_text = row['A']  # Ваш текстовый документ
    target_text = row['C']  # Извлекаемая информация

    formatted_data.append({
        'input_text': input_text,
        'target_text': target_text
    })

formatted_df = pd.DataFrame(formatted_data)

print(formatted_df.head())

                                          input_text  \
0  \nКитайский документ 13-10trans.txt\nГлава I У...   
1  20230731_12345\nКитайский документ 38-15trans....   
2  \nКитайский документ 26-14trans.txt\nГлава I. ...   
3  \nКитайский документ 28-2trans.txt\nГлава I. О...   
4  \nКитайский документ 85-14trans.txt\nГлава I У...   

                                         target_text  
0  Тендер 1: Китайский документ 13-10trans.txt\n\...  
1  Заказчик: [China Engineering Limited (Чайна ин...  
2  Заказчик: [China & China's New Energy Ltd. Dun...  
3  Заказчик: [China Institute of Nuclear] Предмет...  
4  Заказчик: Консорциум обширных ядерных исследов...  


Код с основной моделью и архитектурой

In [9]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

class CustomDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.input_texts = data['input_text']
        self.target_texts = data['target_text']
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Конкатенация входного и целевого текста для обучения в одной последовательности
        concatenated_text = input_text + " " + target_text
        encoding = self.tokenizer.encode_plus(concatenated_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': encoding['input_ids'].squeeze()
        }

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

dataset = CustomDataset(tokenizer, formatted_df)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    dataloader_pin_memory=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=180, training_loss=2.144845241970486, metrics={'train_runtime': 143.5297, 'train_samples_per_second': 5.852, 'train_steps_per_second': 1.254, 'total_flos': 219485306880000.0, 'train_loss': 2.144845241970486, 'epoch': 15.0})

Тот же код что и выше, просто с поиском лучших гиперпараметров

In [ ]:

def model_init():
    return GPT2LMHeadModel.from_pretrained(model_name)

def objective(trial):
    # Определите диапазон гиперпараметров
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 15)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])

    # Определите аргументы обучения с использованием предложенных гиперпараметров
    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        report_to="none"  # Установите в 'none' для предотвращения отправки логов на серверы huggingface
    )

    # Создайте объект Trainer
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=dataset
    )

    # Обучите модель
    trainer.train()

    # Верните метрику, которую вы хотите оптимизировать (например, loss)
    return trainer.evaluate()['eval_loss']

# Создайте объект study и выполните оптимизацию
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Выведите лучшие гиперпараметры
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Загрузка для предсказания

In [10]:
import torch

random_idx = torch.randint(len(dataset), (1,)).item()
input_text = dataset[random_idx]['input_ids'].unsqueeze(0).to(device)
attention_mask = dataset[random_idx]['attention_mask'].unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    outputs = model.generate(input_ids=input_text, attention_mask=attention_mask, max_length=512)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Input text:", dataset.input_texts[random_idx])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Input text: 

Китайский документ 87-11trans.txt
Глава I Уведомление о торгах
Район, в котором осуществляется тендерный проект: провинция Шаньдун
Условия торгов
Проект LOT104H (Система измерения RIC) (номер тендера: CGN-202211140008) утвержден, проект
Источником финансирования является самофинансируемое предприятие, которое было привлечено к участию в конкурсе на участие компании &quot; Чайна Хинси &quot; . В рамках этого проекта уже имеются условия для проведения торгов, и в настоящее время он открыт.
торги.
II. ОБЩИЙ ОБЗОР ПРОЕКТОВ И ОГРАНИЧЕНИЙ
Проект SK расположен в провинции Шаньдун и построен в двух энергоблоках на миллион киловатт-гигаватт, а техническая программа - Huawei.
Программа технической интеграции.
Содержание и сфера охвата тендера: данный тендерный проект разделен на пункты, которые не разделены на тендерные заявки, и этот тендер будет включать в себя:
Оборудование и соответствующее проектирование, упаковка, запасные части SE, расходные материалы, специализированные инс

результат генерации

In [11]:
print("Generated text:", generated_text)

Generated text: дерной документации в режиме онлайн, и для этого проекта не требуется загрузка приложения.
5) Заявители должны завершить регистрацию тендера в течение срока действия тендерной документации, а просроченная система будет автоматически закрыта. В соответствии с вышеуказанными требованиями
Участник торгов, представивший заявку и загрузивший тендерную документацию, не может участвовать в торгах, а участник торгов имеет право отказать в получении тендерной документации, взимаемой, в соответствии с
Обработка недействительных документов.
6) За участие в торгах, которое не было представлено в соответствии с вышеуказанными требованиями, участник торгов берет на себя ответственность.
Представление тендерной документации
Дата представления: 05 декабря 2022 года, 09:00 (Пекинское время)


*%* "Заказчик: Компания ""Чайна Хинси"".

Предмет контракта: Проект LOT104H (Система измерения RIC).

Территория тендера: Провинция Шаньдун, Китай.

Сегменты тендера: #ГражданскаяЯдернаяБезопасност

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
class CustomDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.input_texts = data['input_text']
        self.target_texts = data['target_text']
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]
        source = self.tokenizer.encode_plus(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt", return_attention_mask=True)
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt", return_attention_mask=True)

        # Перемещение тензоров на GPU, если она доступна
        if torch.cuda.is_available():
            source = {k: v.to(device) for k, v in source.items()}
            target = {k: v.to(device) for k, v in target.items()}

        return {
            'input_ids': source['input_ids'].squeeze(),
            'attention_mask': source['attention_mask'].squeeze(),
            'labels': target['input_ids'].squeeze()
        }


model_name = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)
dataset = CustomDataset(tokenizer, formatted_df)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    dataloader_pin_memory=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use th

OutOfMemoryError: ignored

In [ ]:
first_example = formatted_df.iloc[0]
input_text = first_example['input_text']

# Кодирование текста для подачи в модель
encoded_input = tokenizer.encode("extract: " + input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
# Генерация предсказания
with torch.no_grad():
    output = model.generate(encoded_input, max_length=350, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# Декодирование предсказания
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# Вывод входного текста и предсказания
print(f"Входной текст: {input_text}")
print(f"Сгенерированное предсказание: {decoded_output}")

In [ ]:
from torch.utils.data import DataLoader

# Подготовка DataLoader для тестового набора
test_dataset = CustomDataset(tokenizer, test_data) # test_data должен быть вашим тестовым DataFrame
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Функция для оценки модели на тестовом наборе
def evaluate_model(model, test_loader, device):
    model.eval()  # Перевод модели в режим оценки
    all_predictions = []

    with torch.no_grad():  # Отключение вычисления градиентов
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)

            predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            all_predictions.extend(predictions)

    return all_predictions

# Вызов функции оценки
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
predictions = evaluate_model(model, test_loader, device)

# Выводим несколько примеров предсказаний
for i in range(5):
    print(f"Входные данные: {test_data['input_text'].iloc[i]}")
    print(f"Сгенерированный текст: {predictions[i]}")
    print("\n")

NameError: ignored

In [ ]:
# Открываем файл и читаем его как текстовый файл
with open('all_applic.dat', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Разделяем текст с разделителем "*$*"
lines = file_content.split('*$*')

# Загружаем данные в pandas DataFrame
data = pd.DataFrame({'Text': lines})

# Просмотр первых нескольких строк данных
##print(data['Text'].iloc[4])

# Открываем файл и читаем его как текстовый файл
with open('/content/all_meta.dat', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Разделяем текст с разделителем "*$*"
lines = file_content.split('*$*')

# Загружаем данные в pandas DataFrame
meta = pd.DataFrame({'Text': lines})

# Просмотр первых нескольких строк данных
# print(meta['Text'].iloc[4])

In [ ]:
# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Создание объекта Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

# Запуск обучения
trainer.train()

In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Функция для извлечения информации
def extract_information(text, model, tokenizer):
    input_ids = tokenizer.encode("extract: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Загрузка модели и токенизатора
model_name = "cointegrated/rut5-base-absum"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Загрузка данных
with open('all_applic.dat', 'r', encoding='utf-8') as file:
    file_content = file.read()
lines = file_content.split('*$*')
data = pd.DataFrame({'Text': lines})

# Применение функции извлечения информации к первым пяти текстам в DataFrame
for index, row in data.iloc[:5].iterrows():
    extracted_info = extract_information(row['Text'], model, tokenizer)
    data.at[index, 'Extracted_Info'] = extracted_info

# Вывод первых нескольких строк
print(data.head())

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

                                                Text  \
0                                                      
1  \n20230731_12345\nКитайский документ 22-9trans...   
2  \n20230731_12345\nКитайский документ 85-8trans...   
3  \n20230731_12345\nКитайский документ 8-8trans....   
4  \n20230731_12345\nКитайский документ 67-9trans...   

                                      Extracted_Info  
0  Для того, чтобы избавиться от сахара, можно ис...  
1  Совместимая закупка оборудования для замены ко...  
2  Внутренняя Монголия закупила оборудование для ...  
3  Китайская база экологически чистой энергии в С...  
4  Китайская компания Чайна нью-Энтерпрайз и Чайн...  


In [ ]:
print(data['Extracted_Info'].iloc[1])

Совместимая закупка оборудования для замены коробок была разделена на четыре сегмента: 1-й пункт: кузов; 2-й участок: горная гора Гуанси; 3-й пункт: 5-й армейский.


In [ ]:
data['Text'].to_csv('textROSATOm.csv',index=False)

In [ ]:
meta['Text'].iloc[1]

'\nДата выхода: 2023-04-11\nУстановлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы\n(Контингент: CGN-20180529004)\nПервый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян (заявление No CGN-20180529004)\nУстановлено следующее:\n1. Информация о кандидатах:\nHongong Kong\nПобедитель:\nЦены: RMB14 068 380.00\nII. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:\nIII. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА\n.\nНаблюдение за осуществлением данного тендерного проекта /\nIV. Связи\nЗаявители: &quot; Чайна инжиниринг лимитед &quot;\nМестонахождение:/\nОрганизация: двойная\nТелефон: 0755-88616541\nЭлектронная почта: luozhenggang@cgnpc.com.cn\nАгент по торгам: &quot; Чайна инжиниринг лимитед &quot;\nМестонахождение:/\nОрганизация: Худо\nТелефон: 0755-88610728\nЭлектронная почта:huduo@cgnpc.com.cn\nУчастник торгов или его главный руководитель (руководитель проекта): (подпись)\nУчастник торгов или его агент: (глава)\n\n\n'

In [ ]:
x = pd.read_csv('/content/textROSATOm.csv')
x

,Text
0,NaN
1,\n20230731_12345\nКитайский документ 22-9trans...
2,\n20230731_12345\nКитайский документ 85-8trans...
3,\n20230731_12345\nКитайский документ 8-8trans....
4,\n20230731_12345\nКитайский документ 67-9trans...
...,...
1425,\n20230731_12345\nКитайский документ 22-6trans...
1426,\n20230731_12345\nКитайский документ 32-14tran...
1427,\n20230731_12345\nКитайский документ 73-15tran...
1428,\n20230731_12345\nКитайский документ 9-3trans....


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Загрузка модели и токенизатора
model_name = "Natet/rut5_base_sum_gazeta-finetuned"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Функция для суммаризации текста
def summarize_text(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
for index, row in data.head(10).iterrows():
    summary = summarize_text(row['Text'])
    data.at[index, 'Summary'] = summary

In [ ]:
data['Summary'].iloc[1]

'<unk> Китайская Новая энергетическая компания объявила о совместной закупке оборудования для замены коробок в 2023 году. Проект может быть завершен в ближайшие три года.</s>'

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
# Пример токенизации текста в столбце "Unnamed: 0"
data['tokens'] = data['Unnamed: 0'].apply(lambda x: word_tokenize(str(x)))

# Просмотр столбца с токенами
print(data['tokens'].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0                                     [20230731_12345]
1                 [Китайский, документ, 22-9trans.txt]
2                   [Глава, I, Уведомление, о, торгах]
3    [Территория, ,, в, которой, осуществляется, те...
4                                    [Условия, торгов]
Name: tokens, dtype: object
